In [1]:
###Importing the Important Libraries

import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import keras
import os
import random

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
cd '/kaggle/working'

/kaggle/working


In [3]:
mkdir train

In [4]:
mkdir test

In [5]:
cp -a /kaggle/input/intel-image-classification/seg_train/seg_train/forest/. /kaggle/working/train

In [6]:
cp -a /kaggle/input/intel-image-classification/seg_train/seg_train/glacier/. /kaggle/working/train

In [7]:
cp -a /kaggle/input/intel-image-classification/seg_train/seg_train/buildings/. /kaggle/working/train

In [8]:
cp -a /kaggle/input/intel-image-classification/seg_train/seg_train/mountain/. /kaggle/working/train

In [9]:
cp -a /kaggle/input/intel-image-classification/seg_train/seg_train/sea/. /kaggle/working/train

In [10]:
cp -a /kaggle/input/intel-image-classification/seg_train/seg_train/street/. /kaggle/working/train

In [11]:
filenames = os.listdir("/kaggle/working/train")
print(len(filenames))

14034


In [12]:
###Preparing Training Data

##Forest
filenames = os.listdir("/kaggle/input/intel-image-classification/seg_train/seg_train/forest/")
print(len(filenames))

df_forest= pd.DataFrame({
    'filename': filenames,
    'category': 'forest'
})

##Glacier
filenames = os.listdir("/kaggle/input/intel-image-classification/seg_train/seg_train/glacier/")
print(len(filenames))

df_glacier= pd.DataFrame({
    'filename': filenames,
    'category': 'glacier'
})

##Buildings
filenames = os.listdir("/kaggle/input/intel-image-classification/seg_train/seg_train/buildings/")
print(len(filenames))

df_buildings= pd.DataFrame({
    'filename': filenames,
    'category': 'buildings'
})

##Sea
filenames = os.listdir("/kaggle/input/intel-image-classification/seg_train/seg_train/sea/")
print(len(filenames))

df_sea= pd.DataFrame({
    'filename': filenames,
    'category': 'sea'
})

2271
2404
2191
2274


In [13]:
##street

filenames = os.listdir("/kaggle/input/intel-image-classification/seg_train/seg_train/street/")
print(len(filenames))

df_street= pd.DataFrame({
    'filename': filenames,
    'category': 'street'
})

2382


In [14]:
##Mountain

filenames = os.listdir("/kaggle/input/intel-image-classification/seg_train/seg_train/mountain/")
print(len(filenames))

df_mountain= pd.DataFrame({
    'filename': filenames,
    'category': 'mountain'
})

2512


In [15]:
filenames = os.listdir("/kaggle/working/train")
print(len(filenames))

14034


In [16]:
cp -a /kaggle/input/intel-image-classification/seg_test/seg_test/forest/. /kaggle/working/test

In [17]:
cp -a /kaggle/input/intel-image-classification/seg_test/seg_test/glacier/. /kaggle/working/test

In [18]:
cp -a /kaggle/input/intel-image-classification/seg_test/seg_test/buildings/. /kaggle/working/test

In [19]:
cp -a /kaggle/input/intel-image-classification/seg_test/seg_test/mountain/. /kaggle/working/test

In [20]:
cp -a /kaggle/input/intel-image-classification/seg_test/seg_test/sea/. /kaggle/working/test

In [21]:
cp -a /kaggle/input/intel-image-classification/seg_test/seg_test/street/. /kaggle/working/test

In [22]:
filenames = os.listdir("/kaggle/working/test")
print(len(filenames))

3000


In [23]:
###Preparing Val Data

##Forest
filenames = os.listdir("/kaggle/input/intel-image-classification/seg_test/seg_test/forest/")
print(len(filenames))

df_forest_val= pd.DataFrame({
    'filename': filenames,
    'category': 'forest'
})

##Glacier
filenames = os.listdir("/kaggle/input/intel-image-classification/seg_test/seg_test/glacier/")
print(len(filenames))

df_glacier_val= pd.DataFrame({
    'filename': filenames,
    'category': 'glacier'
})

##Buildings
filenames = os.listdir("/kaggle/input/intel-image-classification/seg_test/seg_test/buildings/")
print(len(filenames))

df_buildings_val= pd.DataFrame({
    'filename': filenames,
    'category': 'buildings'
})

##Sea
filenames = os.listdir("/kaggle/input/intel-image-classification/seg_test/seg_test/sea/")
print(len(filenames))

df_sea_val= pd.DataFrame({
    'filename': filenames,
    'category': 'sea'
})

##Mountain
filenames = os.listdir("/kaggle/input/intel-image-classification/seg_test/seg_test/mountain/")
print(len(filenames))

df_mountain_val= pd.DataFrame({
    'filename': filenames,
    'category': 'mountain'
})

##street
filenames = os.listdir("/kaggle/input/intel-image-classification/seg_test/seg_test/street/")
print(len(filenames))

df_street_val= pd.DataFrame({
    'filename': filenames,
    'category': 'street'
})

474
553
437
510
525
501


In [24]:
###Image Hyperparameters

IMAGE_SIZE = 150
IMAGE_WIDTH,IMAGE_HEIGHT = IMAGE_SIZE,IMAGE_SIZE
IMAGE_SIZE = (IMAGE_HEIGHT,IMAGE_WIDTH)
IMAGE_CHANNELS = 3

In [25]:
###Creating the Model

from keras.applications import VGG16

input_shape = (IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_CHANNELS)

model = Sequential()

pre_trained_model = VGG16(input_shape = input_shape , include_top = False , weights = 'imagenet')

for layer in pre_trained_model.layers[:15]:
    layer.trainable = False
for layer in pre_trained_model.layers[15:]:
    layer.trainable = True
    
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = Flatten()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = Dense(6, activation='softmax')(x)


model = Model(pre_trained_model.input, x)

58892288/58889256 [==============================] - 2s 0us/step


In [26]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [27]:
###Concatenating the DataFrames(Train)

frames = [df_forest,df_glacier,df_buildings,df_mountain,df_sea,df_street]
train_df = pd.concat(frames)

In [28]:
###Concatenating the DataFrames(Val)

frames = [df_forest_val,df_glacier_val,df_buildings_val,df_mountain_val,df_sea_val,df_street_val]
validate_df = pd.concat(frames)

In [29]:
print(train_df.shape)
print(validate_df.shape)

(14034, 2)
(3000, 2)


In [30]:
###Hyperparameters

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size= 64
epochs = 20

In [31]:
###Generator

##Train
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "/kaggle/working/train", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

Found 14034 validated image filenames belonging to 6 classes.


In [32]:
##Val 
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/kaggle/working/test", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

Found 3000 validated image filenames belonging to 6 classes.


In [33]:
###Callbacks

learning_rate_reduction = ReduceLROnPlateau(monitor='val_categorical_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

earlystop = EarlyStopping(patience=10)

callbacks = [earlystop, learning_rate_reduction]

In [34]:
###Compiling the Model

from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['categorical_accuracy'])

In [35]:
###Training the Model

history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Epoch 1/20
219/219 [==============================] - 89s 407ms/step - loss: 0.6624 - categorical_accuracy: 0.7564 - val_loss: 0.3609 - val_categorical_accuracy: 0.8903
Epoch 2/20
219/219 [==============================] - 87s 396ms/step - loss: 0.3821 - categorical_accuracy: 0.8724 - val_loss: 0.4597 - val_categorical_accuracy: 0.8917
Epoch 3/20
219/219 [==============================] - 87s 397ms/step - loss: 0.3305 - categorical_accuracy: 0.8888 - val_loss: 0.2573 - val_categorical_accuracy: 0.9118
Epoch 4/20
219/219 [==============================] - 86s 391ms/step - loss: 0.2861 - categorical_accuracy: 0.9037 - val_loss: 0.2579 - val_categorical_accuracy: 0.8924
Epoch 5/20
219/219 [==============================] - 87s 395ms/step - loss: 0.2596 - categorical_accuracy: 0.9122 - val_loss: 0.3076 - val_categorical_accuracy: 0.9172
Epoch 6/20
219/219 [==============================] - 87s 399ms/step - loss: 0.2294 - categorical_accuracy: 0.9199 - val_loss: 0.3663 - val_categorical_acc

In [37]:
###Acc of about 92%